In [ ]:
import hopsworks
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
import os
import numpy as np
import joblib
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# # You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
# Get features (data) from feature store
# bitcoin data
bitcoin_fg = fs.get_feature_group(name="bitcoin", version=1)
print(bitcoin_fg)
query = bitcoin_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="bitcoin",
                                  version=1,
                                  description="Read from bitcoin dataset",
                                  labels=["close"],
                                  query=query)
print(feature_view)

df_bitcoin = bitcoin_fg.read()
df_bitcoin = df_bitcoin.sort_values(by="date")
df_bitcoin.reset_index(drop=True,inplace=True)

print(df_bitcoin)
y_bitcoin = df_bitcoin['close'].values.reshape(-1,1)

In [ ]:
# forecast setting
n_forecast = 90  # length of output sequences (forecast period)
n_lookback = 3*n_forecast  # length of input sequences (lookback period, should be 3 times longer than forecast period)

In [ ]:
# Model Training for bitcoin
scaler_bitcoin = MinMaxScaler(feature_range=(0, 1))
scaler_bitcoin = scaler_bitcoin.fit(y_bitcoin)
y_bitcoin = scaler_bitcoin.transform(y_bitcoin)

X_bitcoin = []
Y_bitcoin = []

for i in range(n_lookback, len(y_bitcoin) - n_forecast + 1):
    X_bitcoin.append(y_bitcoin[i - n_lookback: i])
    Y_bitcoin.append(y_bitcoin[i: i + n_forecast])

X_bitcoin = np.array(X_bitcoin)
Y_bitcoin = np.array(Y_bitcoin)
print(X_bitcoin.shape, Y_bitcoin.shape)

# fit / train the model
model_bitcoin = Sequential()
model_bitcoin.add(LSTM(units=128, return_sequences=True, input_shape=(n_lookback, 1)))
model_bitcoin.add(LSTM(units=64, return_sequences=True))
model_bitcoin.add(LSTM(units=64, return_sequences=False))
model_bitcoin.add(Dense(n_forecast))
model_bitcoin.compile(loss='mean_squared_error', optimizer='adam')
model_bitcoin.fit(X_bitcoin, Y_bitcoin, epochs=20, batch_size=32, verbose=1)

In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()
model_dir="bitcoin_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model_bitcoin, model_dir + "/bitcoin_model.pkl")
# fig.savefig(model_dir + "/wine_confusion_matrix.png")

# # Create an entry in the model registry that includes the model's name, desc, metrics
bitcoin_model = mr.python.create_model(
    name="bitcoin_model",
    # metrics={"accuracy" : metrics['accuracy']},
    # model_schema=model_schema,
    description="bitcoin Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
bitcoin_model.save(model_dir)
